# Batch processing
Tobias Rose 2020

# Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
sns.set()  # set plot styles
import sys
import os
from pathlib import Path
from scipy.signal import convolve
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from tqdm import tqdm

import pandas as pd

# Function definitions

### Aux loading

In [ ]:
def load_auxdata(filename):
    """ Loads .lvd aux data file """
    with open(filename, 'rb') as f:
        # Reset file index
        f.seek(0)
        # Get meta data
        samplingfreq = np.fromfile(f, dtype='>f8', count=1)
        print("Aux sampling frequency = {}Hz".format(samplingfreq))
        n_channels = int(np.fromfile(f, dtype='>f8', count=1))
        print("# channels = {}".format(n_channels))
        timestamp = np.fromfile(f, dtype='>f8', count=1)
        print("timestamp = {}".format(timestamp))
        max_input = np.fromfile(f, dtype='>f8', count=1)
        print("max input = {} V".format(max_input))
        # Read aux data
        auxdata = np.fromfile(f, dtype='>f8')
        n_datapoints = int(auxdata.shape[0]/n_channels)
        print("number of aux datapoints = {}".format(n_datapoints))
        auxdata = np.reshape(auxdata,(n_datapoints,n_channels))
        return auxdata, samplingfreq

### Frame extraction

In [ ]:
def get_frame_times(auxdata, Frames_chan):
    """ extracts frame onset times """
    len_aux = len(auxdata)
    pos = np.argwhere(auxdata[0:,Frames_chan] > 0.75 * np.max(auxdata[range(0,len_aux),Frames_chan ])) # work on diff of indices rather than on raw diff to prevent multi-smaple detection in up/ downstrokes
    diffpos = np.argwhere(np.diff(pos[0:,0]) > 1)
    frame_times = pos[diffpos,0]
    
    if  len(frame_times)==0:
        print('get_frame_times WARNING: no frames found')
        frame_times = 1;
        return frame_times
    
    # find onset of first frame
    pos_first = np.argwhere(auxdata[0:,Frames_chan] < 0.5 * np.max(auxdata[range(0,len_aux), Frames_chan]))
    diffpos_first = np.argwhere(np.diff(pos_first[0:,0]) > 1)
    frame_times = np.append(diffpos_first[0], frame_times)
    
    return frame_times

### CHIRP - stimulus bound extraction

In [ ]:
def get_stimIDs_chirp(auxdata, stimops):
    """ extracts chirp onset times """
    
    Frames_chan = stimops['Frames_chan']
    Stims_chan = stimops['Stims_chan']
    eye1_chan = stimops['eye1_chan']
    eye2_chan = stimops['eye2_chan']
    level = stimops['level']
    minsample_delta = stimops['minsample_delta']
    
    frame_times         = get_frame_times(auxdata,Frames_chan)
    frame_times_level   = frame_times[range(0,len(frame_times),level)]

    
    StimOn = auxdata[frame_times_level, Stims_chan]>0.8

    # generate cleaned eye binaries
    Eye1On = auxdata[frame_times_level,eye1_chan]*-1+ np.max(auxdata[frame_times_level,eye1_chan])>0.8
    Eye2On = auxdata[frame_times_level,eye2_chan]>0.8
    Eye2On[-1] = 1 
    bino = Eye1On == Eye2On

    Eye1On_only = Eye1On != bino
    Eye2On_only = Eye2On != bino
    Eye1On_only[-1] = False
    Eye2On_only[-1] = False
    bino[0] = False
    bino[-1] = False

    # generate cleaned bino binary
    bino_onsets_temp  = np.argwhere(np.diff(np.multiply(bino, 1)) > 0)
    bino_offsets_temp = np.argwhere(np.diff(np.multiply(bino, 1)) < 0)

    bino_onsets  = bino_onsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]
    bino_offsets = bino_offsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]                              

    bino_clean = np.full(( len(frame_times_level)), False) 

    for i in range(len(bino_onsets)):
        bino_clean[range(bino_onsets[i,0,0], bino_offsets[i,0,0])] = True

    # extract chirp stim on and offsets
    chirp_onsets_temp  = np.argwhere(np.diff(np.multiply(StimOn, 1)) > 0)
    chirp_offsets_temp = np.argwhere(np.diff(np.multiply(StimOn, 1)) < 0)

    chirp_on  = np.argwhere(np.diff(chirp_onsets_temp[0:,0]) > minsample_delta) + 1
    chirp_off = np.argwhere(np.diff(chirp_offsets_temp[0:,0]) > minsample_delta)

    chirp_on  = np.append(0, chirp_on)
    chirp_off = np.append(chirp_off, len(chirp_offsets_temp) - 1)

    chirp_onsets  = chirp_onsets_temp[chirp_on]
    chirp_offsets = chirp_offsets_temp[chirp_off]

    ids = { 
    'Ipsi':   [np.intersect1d(chirp_onsets, np.argwhere(Eye1On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye1On_only))],
    'Contra': [np.intersect1d(chirp_onsets, np.argwhere(Eye2On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye2On_only))],
    'Bino':   [np.intersect1d(chirp_onsets, np.argwhere(bino_clean)), np.intersect1d(chirp_offsets, np.argwhere(bino_clean))],
    'FrameTimes_level': frame_times_level,
    'FrameTimes':       frame_times,
    }
    
    return ids
    

# Batch processing

### User-specific folders

In [ ]:
if sys.platform == "darwin":
    ### Mac
    main_root = '/Volumes/archive_bonhoeffer_group$/David Laubender/Data/imaging data/DL_191024_6/ImagingData/' #location of original data
    adata     = '/Volumes/archive_bonhoeffer_group$/David Laubender/adata' #location of saved analyzed data
    ftemp     = '/Users/trose/Data/temp' #fast disk (local ssd for s2p binary files) 
    ftiff     = '/Users/trose/Data/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
elif sys.platform == "win32":
    main_root = 'I:/David Laubender/Data/imaging data/DL_191024_6/ImagingData' #location of original data
    adata     = 'I:/David Laubender/adata' #location of saved analyzed data
    ftemp     = 'C:/temp/trose/suite2ptemp' #fast disk (local ssd for s2p binary files  
    ftiff     = 'C:/temp/trose/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)

In [ ]:
exp = ['62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290']

In [ ]:
exp = ['62284']

In [ ]:
adata_s2ppath = []
adata_roipath = []
aux_files = []
tiff_file =[]

for val in exp:
    s2pdir = list(Path(main_root).rglob('suite2p_exp'+val+'/')) #recursive
    tifffile = list(Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'ImagingData', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.tif')) #recursive search over main_root    
    #print(s2pdir)
    #print(tifffile)
    try:
        adata_s2ppath.append(os.path.join(s2pdir[0], 'suite2p', 'combined'))
        adata_roipath.append(os.path.dirname(s2pdir[0]))
        aux_files.append(*Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'data', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.lvd'))
        tiff_file.append(str(tifffile[0]))
    except:
        print(val + ' not found')

### parse first tiff of exp for imaging specs

In [ ]:
with ScanImageTiffReader(tiff_file[0]) as reader:
    header = (reader.description(0))
    mov_dim = (reader.shape())
    
level = pSI.parse_SI_header_level(header)
zoom = pSI.parse_SI_header_zoom(header)
framerate = pSI.parse_SI_header_FrameRate(header)
channels = pSI.parse_SI_header_Channels(header)
volumes = pSI.parse_SI_header_Volumes(header)
frames = pSI.parse_SI_header_Frames(header)
frames_per_file = pSI.parse_SI_header_FramesPerFile(header)

# account for multilevel acq where frames is 1
if frames < volumes:
    frames = volumes

### extract chirp stim timebase

In [ ]:
aux_filename = str(aux_files[0])

In [ ]:
[auxdata, samplingfreq] = load_auxdata(aux_filename)

In [ ]:
stimops = {
    'Frames_chan': 3,
    'Stims_chan': 7,
    'eye1_chan': 16,
    'eye2_chan': 17,
    'level': level, # extract from SI file in the future
    'minsample_delta': 100 
     }

In [ ]:
ids = get_stimIDs_chirp(auxdata, stimops)

### plot aux_data

#### raw aux plus stimbounds

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.subplot(3,1,1)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['Stims_chan']]), plt.ylabel('Stims_chan') 
plt.vlines(ids['Ipsi'][0],-1,6, 'r')
plt.vlines(ids['Ipsi'][1],-1,6, 'r')
plt.vlines(ids['Contra'][0],-1,6, 'b')
plt.vlines(ids['Contra'][1],-1,6, 'b')
plt.vlines(ids['Bino'][0],-1,6, 'k')
plt.vlines(ids['Bino'][1],-1,6, 'k')
plt.subplot(3,1,2)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye1_chan']]), plt.ylabel('eye1_chan') 
plt.subplot(3,1,3)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye2_chan']]), plt.ylabel('eye2_chan') 
plt.show

# make PSTH

### load traces

In [ ]:
F = np.load(os.path.join(adata_s2ppath[0],'F.npy'))
Fneu = np.load(os.path.join(adata_s2ppath[0],'Fneu.npy'))
spks = np.load(os.path.join(adata_s2ppath[0],'spks.npy'))
stat = np.load(os.path.join(adata_s2ppath[0],'stat.npy'), allow_pickle=True)
ops = np.load(os.path.join(adata_s2ppath[0],'ops.npy'), allow_pickle=True).item()
iscell = np.load(os.path.join(adata_s2ppath[0],'iscell.npy'))

### trash non-cells

In [ ]:
F = F[iscell[:,0]==1,:]
Fneu =Fneu[iscell[:,0]==1,:]
spks = spks[iscell[:,0]==1,:]

### PANDAS: generate seaborn-compatible, stimulus-chopped long-form pandas and plot (maybe not)

In [ ]:
categories = ['Ipsi', 'Contra', 'Bino'] #ids.keys()
dfs = []
dataframe =[]
cell = 1
ncells = F.shape[0]

prestim  = 1 #seconds before stimulus
poststim = 1 #seconds after stimulus

prestim_frames = np.ceil(prestim * (framerate / level))
poststim_frames = np.ceil(prestim * (framerate / level))

In [ ]:
dfs = []
dataframe = []
# allocate PSTH array
maxlength = np.int(np.ceil(np.diff(ids["Bino"][:], axis = 0).mean() + prestim_frames + poststim_frames))

# for cell in tqdm(np.random.randint(0,ncells,5)):
for cell in tqdm(range(5)):
    for category in categories:
        for trial, val in enumerate(ids[category][0]):
            
            aligned_F = np.empty((1,maxlength)).squeeze()
            aligned_F.fill(np.NaN)
            aligned_spks = np.empty((1,maxlength)).squeeze()
            aligned_spks.fill(np.NaN)
            
            slice_range = np.arange(ids[category][0][trial].astype(int)-prestim_frames.astype(int),ids[category][1][trial].astype(int)+poststim_frames.astype(int))

            aligned_F[0:len(slice_range)] = F[cell,slice_range]
            aligned_spks[0:len(slice_range)] = spks[cell,slice_range]
            
            time = np.arange(0, aligned_F.shape[0] * 1 / (framerate / level), 1 / (framerate / level) ) - prestim

            dfs.append(pd.DataFrame({
                                    'cell': cell,
                                    'time': time,
                                    'trial':trial,
                                    'aligned_F': aligned_F,
                                    'aligned_spks': aligned_spks,
                                    'category': category
                                    }))            
dataframe = pd.concat(dfs, axis=0)

### Panda convert to long form

In [ ]:
melted = dataframe.melt(id_vars=['category', 'time', 'trial', 'cell'], value_vars=['aligned_F'])

### Panda sclicing examples

In [114]:
# generating trial numpy array from single cell and condition
arr = melted.loc[(melted.loc[:,'trial'] < 8) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Bino'),'value'].to_numpy()
# reshaping for imshow
arr = arr.reshape(8,maxlen)

In [172]:
# generating trial dataframe from single cell and condtion
subselected = melted.loc[(melted.loc[:,'trial'] < 8) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),['time', 'value', 'trial']]

# reshape ('pivot') to index over time and have single trial columns
subselected = subselected.pivot(index = 'time', columns='trial', values='value')
# generate mean from that
subselected_mean = subselected.mean(axis = 1)

fig = plt.figure()
plt.plot(subselected, 'k')
plt.plot(subselected_mean, 'r')

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
melted[melted['trial'] < 8]

,category,time,trial,cell,variable,value
0,Ipsi,-1.000000,0,0,aligned_F,56.307922
1,Ipsi,-0.866667,0,0,aligned_F,69.356964
2,Ipsi,-0.733333,0,0,aligned_F,62.983654
3,Ipsi,-0.600000,0,0,aligned_F,53.777809
4,Ipsi,-0.466667,0,0,aligned_F,49.278210
...,...,...,...,...,...,...
30355,Bino,32.066667,7,4,aligned_F,52.319675
30356,Bino,32.200000,7,4,aligned_F,53.342979
30357,Bino,32.333333,7,4,aligned_F,58.474155
30358,Bino,32.466667,7,4,aligned_F,67.439957


### Panda plotting examples

In [80]:
fig = plt.figure()
melted.loc[(melted.loc[:,'trial'] < 8) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),'value'].plot()

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
#plt.clf()
fig = plt.figure()
sns.set()
with sns.axes_style('white'):
    plt.imshow(arr, aspect = 'auto',  cmap = 'cubehelix')
    #sns.heatmap(arr, cmap = 'cubehelix')
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
#plt.clf()
g = sns.FacetGrid(melted, col='category', hue='category', row='cell', sharey='row', margin_titles=True)
g.map(sns.lineplot, 'time', 'value', ci='sd')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
# plt.clf()
g = sns.relplot(x="time", y="value",
                 col="category", row = 'cell', hue="category", style="category",
                 kind="line", data=melted, estimator=None, facet_kws={'sharey': True, 'sharex': True})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### arrays and classic plotting

### plot examples

In [ ]:
# show cells
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(stat)

for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
fig = plt.figure(figsize=(7,4))
plt.imshow(im)
plt.tight_layout()
plt.show()

In [ ]:
ops['tau'] = .7

In [ ]:
# show mean image
fig = plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
plt.imshow(ops["meanImg"])
plt.subplot(1,3,2)
plt.imshow(ops["meanImgE"])
plt.subplot(1,3,3)
plt.imshow(ops["Vcorr"])
plt.show

In [ ]:
ops['tau'] = 0.5
bouton = 11;

In [ ]:
efilt = np.exp(- np.linspace(0,50,200) / (ops['tau'] * ops['fs']))
#efilt /= efilt.sum()
sout = convolve(spks[bouton,:], efilt)
sout = sout[:spks.shape[1]]

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(F[bouton]-Fneu[bouton] * .7)
plt.show

In [ ]:
plt.plot(sout)
plt.tight_layout()
plt.show
#plt.plot(F[10:])

In [ ]:
plt.plot(spks[11])
plt.show

In [ ]:
plt.figure(figsize=(7,4))
plt.imshow(spks[:100, :5000], vmax = 3, vmin = -0.5, aspect='auto', cmap = 'gray_r')
plt.title('sample of the neural data matrix')
plt.ylabel('boutons') 
plt.xlabel('time [samples]')

In [ ]:
ops

# ToDo

- batch run different deconvolution settings using this code snippet

In [ ]:
# compute deconvolution
from suite2p import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters

ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff,
       'baseline': baseline, 'sig_baseline': sig_baseline, 'win_baseline': win_baseline}

# load traces and subtract neuropil
F = np.load('F.npy')
Fneu = np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(Fc, ops)

# get spikes
spks = dcnv.oasis(Fc, ops)